<center><h1> Usando Modelos de Deep Learning com apoio do OpenCV </h1></center>
<center><h1> Detecção de Objetos </h1></center>

<h2>1 - As bibliotecas/pacotes pré-requisitos para este workshop são:</h2>

<ul>
    <li>OpenCV</li>
    <li>Matplotlib</li>
    <li>Numpy</li>
</ul>
<h3>Inicialmente, será necessário atualizar a versão do OpenCV para 4.5 ou maior.
<ul>
    <li>Executar a célula abaixo (que contém o comando 'pip install opencv-python --upgrade')</li>
 
</ul>



In [ ]:
!pip install opencv-python --upgrade

<ul>
    <li>Importar a biblioteca do OpenCV e confirmar a versão que está sendo usada é 4.5 ou maior.</li>
 
</ul>


In [ ]:
# Importar o OpenCV
import cv2
print("OpenCV version:", cv2.__version__)

In [ ]:
# Importar as demais bibliotecas/pacotes necessários ao projeto

import matplotlib.pyplot as plt
import numpy as np
import os
import requests
from google.colab.patches import cv2_imshow 

## Acesso ao google drive a partir do colab
from google.colab import drive
#drive.mount('/content/gdrive') 
drive.mount("/content/gdrive", force_remount=True)

## IMPORTANTE:
## Pré-requisitos para a configuração de acesso aos recursos do workshop:
## 1. A pasta 'recursos_workshop' deve ter sido compartilhada com o seu usuário do Google Drive
## 2. O aluno deverá fazer acesso à pasta compartilhada em sua conta de Google Drive
## 3. O aluno deverá criar um atalho (shortcut) para a pasta compartilhada. 
##    Este atalho ficará localizado no próprio drive do aluno ('Meu Drive' ou 'MyDrive') e
##    terá o nome 'recursos_workshop'


## Caminho para a pasta de recursos do workshop
resources_path = "gdrive/MyDrive/recursos_workshop/"

## Caminho para a pasta de modelos de Deep Learning
models_path = resources_path + "modelos_DL/"

## Caminho para a pasta de imagens 
image_dir = resources_path + "imagens"



## 2 - Configurações: diretórios de modelos, imagens, notebooks 
#### definições/configurações de modelos de Deep Learning e pesos pré-treinados: 
<ul>
<li>Estão localizados na pasta 'recursos_workshop' do Google drive</li>
</ul>

#### imagens:
<ul>  
    <li>Também estão no google drive ou, dependendo do caso, serão baixadas da internet</li>
</ul>
    


## 3 - Leitura de imagens e pré-processamento 
### 3.1 - A imagem pode ser lida do disco usando o método "imread" do OpenCV
<ul>
    <li>cv2.imread</li>    
</ul>

### 3.2 - O método "blobFromImage" permite fazer resize, crop, scaling, normalizing, mudar de RGB para BGR.
### 3.3 - Este método produz um BLOB de 4 dimensões 
<ul>
    <li>cv2.dnn.blobFromImage</li>
    <li>cv2.dnn.blobFromImages</li>    
</ul>


## 4 - Frameworks e modelos
### 4.1 - Os seguintes frameworks são suportados pelo módulo <a href="https://github.com/opencv/opencv/tree/master/modules/dnn" target="_blank" rel="noopener noreferrer">DNN</a> do OpenCV

<ul>
<li><a href="http://caffe.berkeleyvision.org/" target="_blank" rel="noopener noreferrer">Caffe</a></li>
<li><a href="https://www.tensorflow.org/" target="_blank" rel="noopener noreferrer">Tensorflow</a></li>
<li><a href="http://torch.ch/" target="_blank" rel="noopener noreferrer">Torch</a></li>
<li><a href="https://pjreddie.com/darknet/" target="_blank" rel="noopener noreferrer">Darknet</a></li>
<li><a href="https://onnx.ai/" target="_blank" rel="noopener noreferrer">ONNX</a></li>
</ul>    


## 5 - Carga em memória de modelos de DNN (a partir de modelo serializado em disco)
### OpenCV usa modelos pré-treinados em datasets com acesso público (por exemplo, ImageNet). Esses modelos são desenvolvidos com o uso de diversos frameworks (Caffe, Tensorflow, Pytorch, etc.)
<ul>
    <li>cv2.dnn.readNetFromCaffe</li>
    <li>cv2.dnn.readNetFromDarknet</li>
    <li>cv2.dnn.readNetFromTensorFlow</li>
    <li>cv2.dnn.readNetFromTorch</li>
    <li>cv2.dnn.readNetFromONNX</li>
</ul>

## 6 - Inferência usando o modelo
### 6.1 - Definindo o input para o modelo e iniciando a inferência

<ul>
<li>setInput(blob)</li>
<li>forward</li>
</ul>

## 7 - Detecção de objetos
### A detecção de objetos envolve a classificação e localização dos diferentes objetos contidos em uma imagem, com base em um conjunto pré-definido de classes.
### Logo abaixo, iremos definir algumas funções de apoio para as nossas tarefas.
### Utilizando o modelo Faster-RCNN (desenvolvido com uso do framework Tensorflow) efetuar a tarefa de detecção de objetos nas imagens definidas abaixo e apresentar os resultados


### 7.1 - Carregar em memória o modelo, a partir dos arquivos 'model' e 'config'

In [ ]:
def getNet(faster_rcnn_dir, fmodel, fconfig):
    model = os.path.join(faster_rcnn_dir, fmodel) # binary protobuf description of the network architecture
    config = os.path.join(faster_rcnn_dir, fconfig) # .pbtxt file that contains text graph definition in protobuf format.
    net = cv2.dnn.readNetFromTensorflow(model, config)
    return net

### 7.2 - Setar na variável CLASSES os nomes das classes que foram utilizadas no treinamento deste modelo

In [ ]:
# Classes: MS-COCO dataset com 80 classes
CLASSES = ["person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat",
           "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
           "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack",
           "umbrella","handbag","tie","suitcase","frisbee","skis","snowboard",
           "sports ball","kite","baseball bat","baseball glove","skateboard",
           "surfboard","tennis racket","bottle","wine glass","cup","fork","knife",
           "spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot",
           "hot dog","pizza","donut","cake","chair","sofa","pottedplant","bed",
           "diningtable","toilet","tvmonitor","laptop","mouse","remote","keyboard",
           "cell phone","microwave","oven","toaster","sink","refrigerator","book","clock",
           "vase","scissors","teddy bear","hair drier","toothbrush"]

# Definimos aqui um array de colorações para ser usado no desenho das bounding boxes
COLORS = np.random.uniform(245, 255, size=(len(CLASSES), 3)) #np.random.uniform(0, 255, size=(len(CLASSES), 3))


### 7.3 - Usando OpenCV, efetuar a leitura da imagem contida no path/arquivo 'filePath'

In [ ]:
def imageResize(frame, maxH):
    h, w = frame.shape[:2] # obter a altura e largura do frame
    if maxH < h:
        aspect_ratio = w/h
        blob_height = maxH 
        blob_width = int(blob_height * aspect_ratio)
        dsize = (blob_width, blob_height) # keep the frame's original aspect ratio
        return cv2.resize(frame, dsize)
    return frame

def readImage(filePath):
    frame = cv2.imread(filePath)
    heightMax = 1600  # max height used here to do a imshow inside Colab
    out = imageResize(frame, heightMax)
    return out

### 7.4 - Obter o blob a partir da imagem lida pelo OpenCV, aplicando pré-processamento. O tipo de pré-processamento a ser utilizado irá depender de cada modelo de rede utilizado.

In [ ]:
def getBlobFromFrame(frame, H, W, swapRB, crop):
    
    blob = cv2.dnn.blobFromImage(frame, size=(H,W), swapRB=swapRB, crop=crop)

    return blob

### 7.5 - Definir a imagem lida como entrada para o processamento do modelo

In [ ]:
def setNetInput(net, blob):
    #set input 
    net.setInput(blob)
    return net   

### 7.6 - Processar a inferência neste modelo, usando esta imagem 

In [ ]:
# O blob da imagem a ser usado como input para o modelo realizar a inferência
# precisa já ter sido definido antes de acionar esta função
def processInference(net):
    #forward
    predictions = net.forward()
    return predictions

### 7.7 - Efetuar a detecção de objetos (inferência) usando um modelo já carregado em memória

In [ ]:
def getBlobAndDetectObjects(net, frame, H, W, swapRB, crop, confidence, threshold):               
               
    boxes = []
    confidences = []
    
    height = frame.shape[0]
    width = frame.shape[1]
    
    blob = getBlobFromFrame(frame, H, W, swapRB, crop)

    net = setNetInput(net, blob)
    
    detections = processInference(net)

    classes = []
    
    """
    """
    for detection in detections[0, 0, :, :]:
        class_id = int(detection[1])
        
        score = float(detection[2])

        if score > confidence:
            left = detection[3] * width
            top = detection[4] * height
            right = detection[5] * width
            bottom = detection[6] * height
            box = [int(left), int(top), int(right), int(bottom)]

            confidences.append(score)
            classes.append(class_id)
            boxes.append(box)
    
    
    #using cv2 non maximum suppression
    nms_inds = cv2.dnn.NMSBoxes(boxes, confidences, confidence, threshold)
    
    nms_inds1 = [i[0] for i in nms_inds] # get the indexes
    
    filtered_boxes =  np.array(boxes)[nms_inds1]
    probs =  np.array(confidences)[nms_inds1]
   
 
    
    final_boxes = []
    final_confidences = []
    final_classes = []

    for index, box in enumerate(filtered_boxes):

        startX, startY, endX, endY = box
        box = [int(startX), int(startY), int(endX), int(endY)]

        final_boxes.append(box)
        final_confidences.append(probs[index])
        final_classes.append(classes[index])   ###check this lps


    return final_boxes, final_confidences, final_classes
    


### 7.8 - Desenhar na imagem os marcadores (bounding boxes) para cada uma das classes detectadas

In [ ]:
def drawBoxes(frame, boxes, confidences, classes):
    for i in range(0, len(boxes)):
        ind = classes[i] - 1
        (startX, startY, endX, endY) = boxes[i]
        y = startY - 15 if startY - 15 > 15 else startY + 15
        text = "{}: {:.2f}%".format(CLASSES[ind], confidences[i] * 100)    
        cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[i], 2)
        cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLORS[ind], 2)
    return frame

### 7.9 - Exibir a imagem usando OpenCV

In [ ]:
def displayImage(image, imgTitle="Image"): 
    cv2_imshow(image)


### 7.10 - Efetuar o processo de detecção de objetos utilizando o modelo, uma imagem lida via OpenCV, parâmetros de pré-processamento da imagem e os limites de confiança e threshold

In [ ]:
def procDetect(image_dir, image_file, net, H, W, swapRB, crop, confidence, threshold):
  # Read img from disk
  frame = readImage(os.path.join(image_dir, image_file))

  boxes, confidences, classes = getBlobAndDetectObjects(net, frame, H, W, swapRB, crop, confidence, threshold)

  # Desenhar os marcadores (bounding boxes) para cada uma das classes detectadas 
  frame = drawBoxes(frame, boxes, confidences, classes)

  # exibir a imagem
  displayImage(frame)


### 7.11 - Acionamento do processo de detecção de objetos utilizando as funções de apoio
#### 7.11.1 - Definições de configuração: diretórios de modelos, imagens,  arquivos de modelos e parâmetros de pré-processamento de imagens 


In [ ]:
# diretório contendo os arquivos do modelo Faster-RCNN 
faster_rcnn_dir = models_path + 'Faster_RCNN/faster_rcnn_inception_v2_coco_2018_01_28'

# binary protobuf description of the network architecture
fmodel = 'frozen_inference_graph.pb' 

# .pbtxt file that contains text graph definition in protobuf format.
fconfig =  'graph.pbtxt'

H = 300  # height
W = 300  # width
swapRB=True
crop=False
confidence = 0.4 # só serão aceitas detecções maiores que este valor
threshold = 0.5  # valor a ser aplicado como threshold no non-max suppression
                 # são aceitas bounding boxes com overlapping maior que este valor

##

### 7.12 - Efetuar a carga do modelo em memória

In [ ]:
net = getNet(faster_rcnn_dir, fmodel, fconfig) 


### 7.13 - Efetuar a detecção processo para uma imagem

In [ ]:
image_file = 'horses.jpg'

procDetect(image_dir, image_file, net, H, W, swapRB, crop, confidence, threshold)
 

### Atividade para o aluno
#### 7.14 - Exercício: utilizando o modelo já carregado, efetuar a detecção de objetos para as seguintes imagens
<ul>
<li>pexels-engin-akyurt-1769271.jpg</li>
<li>giraffe.jpg</li>
</ul>

### 7.15 - Usando outros modelos para a detecção de objetos

#### 7.15.1 - Utilizando o modelo MobileNetSSD (desenvolvido com uso do framework Caffe) efetuar a tarefa de detecção de objetos nas imagens definidas abaixo e apresentar os resultados

In [ ]:
# diretório contendo os arquivos do modelo MobileNetSSD 
mobile_ssd_dir = models_path + 'Mobile_SSD'  

# MobileNet-SSD 
# MobileNetSSD_deploy.caffemodel: the model
# MobileNetSSD_deploy.prototxt: the text file that describes the model's parameters

fmodel_file = 'MobileNetSSD_deploy.caffemodel' # the pre-trained model
fconfig =  'MobileNetSSD_deploy.prototxt' # the text file that describes the model's parameters


#### 7.15.2 - Classes - Dataset Pascal VOC - com 20 classes

In [ ]:
class_labels = ["background", 'airplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'dining table', 'dog',
    'horse', 'motorbike', 'person', 'potted plant', 'sheep',
    'sofa', 'train', 'TV or monitor']


#### 7.15.3 - Fazer a carga do modelo em memória

In [ ]:
def getCaffeModel(mobile_ssd_dir, fconfig, fmodel_file):
    model_file = os.path.join(mobile_ssd_dir, fmodel_file) # the pre-trained model
    config = os.path.join(mobile_ssd_dir, fconfig) # the text file that describes the model's parameters
    model = cv2.dnn.readNetFromCaffe(config, model_file)
    return model

#### 7.15.4 -  Efetuar o pré-processamento da imagem lida e obter o blob que será processado pelo modelo - método A

In [ ]:
def getBlobFromImageMobileSSD_pre_A(frame):
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    return blob

#### 7.15.5 - Efetuar o pré-processamento da imagem lida e obter o blob que será processado pelo modelo - método B

In [ ]:
def getBlobFromImageMobileSSD_pre_B(frame):
    h, w = frame.shape[:2]
    aspect_ratio = w/h
    blob_height = 300
    blob_width = int(blob_height * aspect_ratio)
    blob_size = (blob_width, blob_height) # keep the frame's original aspect ratio
    color_scale = 1.0/127.5  ## 0.007843
    average_color = (127.5, 127.5, 127.5)
    blob = cv2.dnn.blobFromImage(frame, scalefactor=color_scale, size=blob_size,  mean=average_color)
    return blob

#### 7.15.6 - Definir o blob como dado de entrada para o modelo e efetuar a detecção de objetos

In [ ]:
def modelSetInputAndDetectObjects(model, blob):
    model.setInput(blob)
    results = model.forward()
    return results

#### 7.15.7 - Para todas as detecções com índice de confiança maior que o limite informado, desenhar as correspondentes 'bounding-boxes' com seus índices de confiança

In [ ]:
def getBoxesAndConfidences(iframe, results, labels, conf_threshold):
    h, w = iframe.shape[:2]
    frame = iframe.copy()
    for object in results[0, 0]:
        confidence = object[2]
        if confidence > conf_threshold:

            # Get the object's coordinates.
            x0, y0, x1, y1 = (object[3:7] * [w, h, w, h]).astype(int)

            # Get the classification result.
            id = int(object[1])
            label = labels[id] ## id - 1
            # Draw a bounding box
            cv2.rectangle(frame, (x0, y0), (x1, y1),
                          (255, 255, 0), 2)

            # Inform class and confidence 
            yt = y0 - 15 if y0 - 15 > 15 else y0 + 15
            text = '%s (%.1f%%)' % (label, confidence * 100.0)
            # cv2.putText(frame, text, (x0, y0 - 20),
            #    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2) 
            cv2.putText(frame, text, (x0, yt),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2) 
    return frame

#### 7.15.8 - Processar a detecção e exibir resultados

In [ ]:
def detectAndShowResults(image_dir, image_file, model, class_labels, conf_threshold):

    # Read img from disk
    frame = readImage(os.path.join(image_dir, image_file))

    # get blob method 1 - blob_A
    blob_A = getBlobFromImageMobileSSD_pre_A(frame)

    # get blob method 2 - blob_B
    blob_B = getBlobFromImageMobileSSD_pre_B(frame)

    # aqui, vamos ver os resultados para dois tipos diferentes de pré-processamento da imagem
    results_A = modelSetInputAndDetectObjects(model, blob_A)
    results_B = modelSetInputAndDetectObjects(model, blob_B)
 
    # put results in the frames 
    frame_A = getBoxesAndConfidences(frame, results_A, class_labels, conf_threshold)    
    frame_B = getBoxesAndConfidences(frame, results_B, class_labels, conf_threshold)    
    
    # show results
    print("\nPre-processamento A")
    displayImage(frame_A, "Pre-proc A")
    print("\nPre-processamento B")
    displayImage(frame_B, "Pre-Proc B")

    

#### 7.15.8 - O processo de deteção: carregar o modelo, obter a imagem a ser processada, realizar a inferência (deteção de objetos) e exibir os resultados

In [ ]:
model = getCaffeModel(mobile_ssd_dir, fconfig, fmodel_file) 
conf_threshold = 0.5   

image_file = 'dog.jpg'

detectAndShowResults(image_dir, image_file, model, class_labels, conf_threshold)
    

### Atividade para o aluno
#### 7.15.9 - Exercício: utilizando o modelo já carregado, efetuar a detecção de objetos para as seguintes imagens
<ul>
<li>person.jpg</li>
<li>giraffe.jpg</li>
</ul>

### Atividade para o aluno
#### 7.15.10 - Utilizando o modelo  Faster-RCNN ResNet-50  (desenvolvido com uso do framework Tensorflow) efetuar a tarefa de detecção de objetos nas imagens definidas abaixo e apresentar os resultados
<ul>
<li>horses.jpg</li>
<li>giraffe.jpg</li>
<li>dog.jpg</li>
</ul>


In [ ]:
# diretório contendo os arquivos do modelo Faster-RCNN ResNet-50 
faster_rcnn_resnet50_dir = models_path + 'Faster_RCNN/faster_rcnn_resnet50_coco_2018_01_28'


fmodel = 'frozen_inference_graph.pb' # the pre-trained model
fconfig = 'faster_rcnn_resnet50_coco_2018_01_28.pbtxt' # the text file that describes the model's parameters

# lembrando que usaremos as mesmas classes do caso de uso anterior com a rede faster_rcnn (treinada com o dataset coco)
# As classes estão definidas na variável CLASSES

H = 300  # height
W = 300  # width
swapRB=True
crop=False
confidence = 0.4 # só serão aceitas detecções maiores que este valor
threshold = 0.5  # valor a ser aplicado como threshold no non-max suppression
                 # são aceitas bounding boxes com overlapping maior que este valor

##

In [ ]:
# implementar a chamada à função getNet


#### 7.15.11 - Utilizando o modelo YOLOV3 (desenvolvido com uso do framework Darknet) efetuar a tarefa de detecção de objetos nas imagens definidas abaixo e apresentar os resultados

In [ ]:
# diretório contendo os arquivos do modelo YOLOV3 
yolov3_dir = models_path +  'Darknet/darknet_detect/yolov3'

yolov3_cfg = 'yolov3.cfg'
yolov3_weights = 'yolov3.weights'

# classes
class_dir  = models_path + 'Darknet/darknet_detect'
class_file = "coco.names"

#### 7.15.12 - Carga das classes

In [ ]:
# carga das classes
classes = open(os.path.join(class_dir, class_file)).read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

#### 7.15.13 - Carga do modelo em memória

In [ ]:
def loadYOLOModel(yolo_dir, yolo_cfg, yolo_weights):
    cfg = os.path.join(yolo_dir, yolo_cfg)
    weights = os.path.join(yolo_dir, yolo_weights)
    dnet = cv2.dnn.readNetFromDarknet(cfg, weights)
    dnet.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    return dnet

def getYOLONetAndLN(yolo_dir, yolo_cfg, yolo_weights):
    # load YOLO net
    net = loadYOLOModel(yolo_dir, yolo_cfg, yolo_weights)
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return net, ln    

### 7.15.14 - Obter o blob a partir da imagem lida pelo OpenCV, aplicando pré-processamento. O tipo de pré-processamento a ser utilizado irá depender de cada modelo de rede utilizado.

In [ ]:
def getBlob(img, H=416, W=416):
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (H, W), swapRB=True, crop=False)
    return blob

### 7.15.15 - Definir a imagem lida como entrada para o processamento do modelo e processar a inferência neste modelo, usando a imagem informada como entrada do processo

In [ ]:
def netProcessImg(blob, net, ln):
    net.setInput(blob)    
    results = net.forward(ln)
    return results

### 7.15.16 - Desenhar na imagem os marcadores (bounding boxes) para cada uma das classes detectadas

In [ ]:
def post_process(img, outputs, conf):
    H, W = img.shape[:2]

    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        scores = output[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > conf:
            x, y, w, h = output[:4] * np.array([W, H, W, H])
            p0 = int(x - w//2), int(y - h//2)
            p1 = int(x + w//2), int(y + h//2)
            boxes.append([*p0, int(w), int(h)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            

    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            
            
    return img

### 7.15.17 - Define o processo de detecção usando as funções de apoio defindas acima

In [ ]:
def detectObjectsInImage(image_dir, image_file, net, ln, H=416, W=416):
    img = readImage(os.path.join(image_dir, image_file))  
    blob = getBlob(img, H, W)
    outputs = netProcessImg(blob, net, ln)
    outputs = np.vstack(outputs)    
    img = post_process(img, outputs, 0.5)
    displayImage(img)

### 7.15.18 - Executar o processo de detecção inicialmente carregando o modelo YOLOV3 em memória

In [ ]:
# Obter a net YOLOV3
net, ln = getYOLONetAndLN(yolov3_dir, yolov3_cfg, yolov3_weights)

In [ ]:
image_file = 'giraffe.jpg'
detectObjectsInImage(image_dir, image_file, net, ln)


### Atividade para o aluno
#### 7.15.19 - Utilizando o modelo  YOLOV3  (desenvolvido com uso do framework Darknet) efetuar a tarefa de detecção de objetos nas imagens definidas abaixo e apresentar os resultados
<ul>
<li>dog.jpg</li>
<li>horses.jpg</li>
<li>person.jpg</li>
</ul>


### Atividade para o aluno
#### 7.15.20 - Utilizando o modelo YOLOV4 (desenvolvido com uso do framework Darknet) efetuar a tarefa de detecção de objetos nas imagens definidas abaixo e apresentar os resultados
<ul>
<li>horses.jpg</li>
<li>giraffe.jpg</li>
<li>dog.jpg</li>
<li>pexels-engin-akyurt-1769271.jpg</li>
<li>pexels-mandie-inman-896567.jpg</li>
</ul>

In [ ]:
# diretório contendo os arquivos do modelo YOLOV4 
yolov4_dir = models_path +  'Darknet/darknet_detect/yolov4'

yolov4_cfg = 'yolov4.cfg'
yolov4_weights = 'yolov4.weights'


In [ ]:
# Obter a net YOLOV4 - chamar a função getYOLONetAndLN


### Atividade para o aluno
#### 7.15.21 - Utilizando o modelo YOLOV4-P6 (desenvolvido com uso do framework Darknet) efetuar a tarefa de detecção de objetos nas imagens definidas abaixo e apresentar os resultados
<ul>
<li>dog.jpg</li>
<li>pexels-mandie-inman-896567.jpg</li>
</ul>

In [ ]:
# diretório contendo os arquivos do modelo YOLOV4-P6
yolov4p6_dir = models_path +  'Darknet/darknet_detect/yolov4-p6'

yolov4p6_cfg = 'yolov4-p6.cfg'
yolov4p6_weights = 'yolov4-p6.weights'



In [ ]:
# Obter a net YOLOV4-P6 - chamar a função getYOLONetAndLN


### Atividade para o aluno
#### 7.15.22 - Ao executar a atividade acima, é possível que tenha ocorrido um erro:
#### ...\modules\dnn\src\layers\concat_layer.cpp:102:  error: (-201:Incorrect size of input array) Inconsistent shape for ConcatLayer  in function 'cv::dnn::ConcatLayerImpl::getMemoryShapes'

#### Execute novamente essa atividade informando o valor 1280 para os parâmetros de altura e largura da imagem ao chamar a função detectObjectsInImage




### Atividade para o aluno
#### 7.15.23 - Questões
<ul>
<li>Por que ocorre erro se não informarmos o valor 1280 para altura e largura nos testes realizados no item 7.15.21?</li>
<li>Com base nos resultados dos testes realizados para as diferentes imagens, é possível dizer se algum dos modelos vistos é melhor que os demais?</li>
</ul>

### Atividade para o aluno
#### 7.15.24 - Atividades extras
<ul>
<li> Utilizando um dos modelos de detecção vistos até aqui, escolhido a seu critério, efetuar a detecção para todos os arquivos de imagens contidos na pasta (de imagens). O seu código deverá gerar um relatório onde cada linha será composta por: nome do arquivo de imagem, número de objetos detectados, nome da classe 1, nome da classe 2, ..., nome da classe 'n'.
Caso uma classe ocorra mais de uma vez para uma dada imagem, informe o nome dessa classe apenas uma vez.

Exemplo:

dog.jpg, 3, truck, dog, bicycle

horses.jpg, 7, horse

pexels-engin-akyurt-1769271.jpg, 1, bird


<li>Melhorar a atividade descrita acima, gravando em outra pasta de sua escolha novos arquivos contendo as imagens resultantes da detecção (com as 'bounding-boxes',etc.). Cada arquivo gerado terá como nome a classe detectada com o maior índice de confiabilidade obtido, se houver empates escolha qualquer uma das melhores colocadas. Por exemplo, o arquivo pexels-engin-akyurt-1769271 apresenta uma detecção de bird. Logo, você deverá gravar a imagem gerada com as detecçoes (bounding-boxes) no novo arquivo bird.jpg. Verifique antes se já não existe um arquivo com o mesmo nome na pasta. Se ocorrer essa situação, adicionar um sufixo sequencial ao nome do arquivo ('_n'): bird_1.jpg, bird_2.jpg, etc. 
</li>
</ul>

#### Para ajudar no desenvolvimento dessas atividades, apresento abaixo exemplos de código para criar uma pasta, gravar e ler um arquivo no ambiente do Google Colab. Os arquivos serão armazenados no seu Google Drive.

In [ ]:
mydrivepath = "gdrive/MyDrive"

In [ ]:
outimage = 'opencv_workshop_saida_imagens'
outdir = os.path.join(mydrivepath, outimage)
if not os.path.exists(outdir):
    os.mkdir(outdir)


In [ ]:
# Write line to file
file = 'Example1.txt'
opfile = os.path.join(outdir, file)  
with open(opfile, 'w') as wfile:
    wfile.write("This is line Example1.txt")

In [ ]:
# Read file
with open(opfile, 'r') as rfile:
    print(rfile.read())